In [1]:
import ee
import pandas as pd
from prophet import Prophet
import streamlit as st
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import folium
import geemap

In [2]:
# Earth Engine API 초기화
ee.Initialize()

In [3]:
aoi = ee.Geometry.Polygon(
    [[[126.19653266055587, 36.853168924185226],
      [126.19653266055587, 36.83836240145186],
      [126.21259510596076, 36.83836240145186],
      [126.21259510596076, 36.853168924185226],
      [126.19653266055587, 36.853168924185226]]]
)

In [4]:
start_date = '2021-01-01'
end_date = '2023-10-31'

In [5]:
sentinel2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .first()

In [6]:
def calculate_moisture(img):
    moisture = img.normalizedDifference(['B8A', 'B11'])
    return img.addBands(moisture.rename('moisture'))

def calculate_NDWI(img):
    NDWI = img.normalizedDifference(['B3', 'B8'])
    return img.addBands(NDWI.rename('NDWI'))

def water_bodies_index(img):
    moisture = img.select('moisture')
    NDWI = img.select('NDWI')
    water_bodies = NDWI.subtract(moisture).divide(NDWI.add(moisture))
    return img.addBands(water_bodies.rename('water_bodies'))

def calculate_water_plants(img):
    water_plants = img.normalizedDifference(['B5', 'B4'])
    NIR2 = img.expression(
        'B4 + (B11 - B4) * ((832.8 - 664.6) / (1613.7 - 664.6))',
        {
            'B4': img.select('B4'),
            'B11': img.select('B11')
        }
    )
    FAI = img.select('B8').subtract(NIR2)
    
    # # Calculate underwater vegetation index
    # UVI = img.normalizedDifference(['B3', 'B5'])
    
    # return img.addBands(water_plants.rename('water_plants')).addBands(FAI.rename('FAI')).addBands(UVI.rename('UVI'))
    return img.addBands(water_plants.rename('water_plants')).addBands(FAI.rename('FAI'))

def calculate_wevi(img):
    # WEVI 계산: (B3 - B4) / (B3 + B4 - B2)
    green = img.select('B3')
    red = img.select('B4')
    blue = img.select('B2')

    wevi = green.subtract(red).divide(green.add(red).subtract(blue))
    return img.addBands(wevi.rename('WEVI'))

def calculate_wavi(img, alpha=1, beta=0.1):
    # WAVI 계산: (B8 - B4) / (B8 + B4 - α * B2 + β)
    NIR = img.select('B8')  # Near-Infrared
    RED = img.select('B4')  # Red
    BLUE = img.select('B2')  # Blue

    wavi = NIR.subtract(RED).divide(NIR.add(RED).subtract(BLUE.multiply(alpha)).add(beta)).rename('WAVI')
    return img.addBands(wavi)

def calculate_ndvi(img):
    # NDVI 계산: (B8 - B4) / (B8 + B4)
    NIR = img.select('B8')  # Near-Infrared
    RED = img.select('B4')  # Red

    ndvi = NIR.subtract(RED).divide(NIR.add(RED)).rename('NDVI')
    return img.addBands(ndvi)

def cloud_mask(img):
    bRatio = img.normalizedDifference(['B3', 'B4'])
    NDGR = img.normalizedDifference(['B3', 'B4'])
    mask = bRatio.gt(1).Or(bRatio.gt(0).And(NDGR.gt(0)))
    return img.updateMask(mask.Not())
# def mask_water_bodies(img):
#     # 'water_bodies' 밴드 선택
#     # water_bodies = img.select('water_bodies')
#     ndvi = img.select('NDVI')

#     # 수역을 나타내는 마스크 생성 (예: 'water_bodies' 값이 0보다 큰 픽셀)
#     water_mask = ndvi.lt(0.8)
#     # 마스크를 적용하여 수역만을 포함하는 이미지 반환
#     return img.updateMask(water_mask)

def process_image(img):
    img = calculate_moisture(img)
    img = calculate_NDWI(img)
    img = water_bodies_index(img)
    img = calculate_wavi(img)
    img = calculate_wevi(img)
    img = calculate_ndvi(img)
    # img = cloud_mask(img)  # 구름 마스킹은 현재 주석 처리되어 있습니다.
    return img
# 신두리사구의 좌표를 사용하여 영역을 정의
# 신두리사구의 좌표를 사용하여 영역을 정의
sinduri_location = ee.Geometry.Polygon([
    [128.913574, 35.024936],
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
])
# 예제 이미지에 함수 적용 (이미지 ID는 적절하게 변경해야 함)
example_image_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2020-01-01', '2020-03-30')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
    .filterBounds(aoi)

# ImageCollection에서 첫 번째 이미지 선택
example_image = example_image_collection.first()  # 첫 번째 이미지를 선택

processed_image = process_image(example_image)
# 지도 생성 및 시각화
# 처리된 이미지에 수역 마스킹 적용
processed_image = cloud_mask(processed_image)

Map = geemap.Map()
Map.centerObject(aoi, 14)  # 신두리사구를 중심으로 설정

# 육지를 마스킹하기 위한 조건 설정
# water_mask = processed_image.select('UVI').gt(0.5)  # 예시로 0.5보다 큰 값인 픽셀을 수상으로 간주

# 처리된 이미지를 지도에 추가 (수중식물 지수 사용) - 수상만 표시
uvi_params = {
    'bands': ['WEVI'],  # UVI 밴드만 사용
    'min': -0.4, # 수중식물 지수의 최소값
    'max': 1,   # 수중식물 지수의 최대값
    # 'palette': ['purple', 'blue', 'green', 'yellow', 'red']  # 색상 팔레트 설정
    'palette': ['red','yellow','green','blue','purple']  # 색상 팔레트 설정
}
Map.addLayer(processed_image, uvi_params, 'Processed Image (FAI)')

# # 다른 밴드도 추가할 경우 (False Color Composite 사용) - 육지 마스킹 적용
# false_color_params = {
#     'bands': ['B8', 'B4', 'B3'],  # B8, B4, B3 밴드를 사용하여 가짜색 합성
#     'min': 0,
#     'max': 3000
# }
# Map.addLayer(processed_image.updateMask(water_mask), false_color_params, 'Processed Image (False Color - Water Only)')

# 지도를 화면에 표시
Map

Map(center=[36.84576569399065, 126.2045638832557], controls=(WidgetControl(options=['position', 'transparent_b…

In [8]:
Map

Map(center=[36.75349996375475, 126.52550000000431], controls=(WidgetControl(options=['position', 'transparent_…

In [11]:
ee.Initialize()

In [3]:
def calculate_moisture(img):
    moisture = img.normalizedDifference(['B8A', 'B11'])
    return img.addBands(moisture.rename('moisture'))

def calculate_NDWI(img):
    NDWI = img.normalizedDifference(['B3', 'B8'])
    return img.addBands(NDWI.rename('NDWI'))

def water_bodies_index(img):
    moisture = img.select('moisture')
    NDWI = img.select('NDWI')
    water_bodies = NDWI.subtract(moisture).divide(NDWI.add(moisture))
    return img.addBands(water_bodies.rename('water_bodies'))

def calculate_water_plants(img):
    water_plants = img.normalizedDifference(['B5', 'B4'])
    NIR2 = img.expression(
        'B4 + (B11 - B4) * ((832.8 - 664.6) / (1613.7 - 664.6))',
        {
            'B4': img.select('B4'),
            'B11': img.select('B11')
        }
    )
    FAI = img.select('B8').subtract(NIR2)
    
    # # Calculate underwater vegetation index
    # UVI = img.normalizedDifference(['B3', 'B5'])
    
    # return img.addBands(water_plants.rename('water_plants')).addBands(FAI.rename('FAI')).addBands(UVI.rename('UVI'))
    return img.addBands(water_plants.rename('water_plants')).addBands(FAI.rename('FAI'))

def calculate_wevi(img):
    # WEVI 계산: (B3 - B4) / (B3 + B4 - B2)
    green = img.select('B3')
    red = img.select('B4')
    blue = img.select('B2')

    wevi = green.subtract(red).divide(green.add(red).subtract(blue))
    return img.addBands(wevi.rename('WEVI'))

def calculate_wavi(img, alpha=1, beta=0.1):
    # WAVI 계산: (B8 - B4) / (B8 + B4 - α * B2 + β)
    NIR = img.select('B8')  # Near-Infrared
    RED = img.select('B4')  # Red
    BLUE = img.select('B2')  # Blue

    wavi = NIR.subtract(RED).divide(NIR.add(RED).subtract(BLUE.multiply(alpha)).add(beta)).rename('WAVI')
    return img.addBands(wavi)

def calculate_ndvi(img):
    # NDVI 계산: (B8 - B4) / (B8 + B4)
    NIR = img.select('B8')  # Near-Infrared
    RED = img.select('B4')  # Red

    ndvi = NIR.subtract(RED).divide(NIR.add(RED)).rename('NDVI')
    return img.addBands(ndvi)

def cloud_mask(img):
    bRatio = img.normalizedDifference(['B3', 'B4'])
    NDGR = img.normalizedDifference(['B3', 'B4'])
    mask = bRatio.gt(1).Or(bRatio.gt(0).And(NDGR.gt(0)))
    return img.updateMask(mask.Not())
# def mask_water_bodies(img):
#     # 'water_bodies' 밴드 선택
#     # water_bodies = img.select('water_bodies')
#     ndvi = img.select('NDVI')

#     # 수역을 나타내는 마스크 생성 (예: 'water_bodies' 값이 0보다 큰 픽셀)
#     water_mask = ndvi.lt(0.8)
#     # 마스크를 적용하여 수역만을 포함하는 이미지 반환
#     return img.updateMask(water_mask)

def process_image(img):
    img = calculate_moisture(img)
    img = calculate_NDWI(img)
    img = water_bodies_index(img)
    img = calculate_wavi(img)
    img = calculate_wevi(img)
    img = calculate_ndvi(img)
    # img = cloud_mask(img)  # 구름 마스킹은 현재 주석 처리되어 있습니다.
    return img
# 신두리사구의 좌표를 사용하여 영역을 정의
# 신두리사구의 좌표를 사용하여 영역을 정의
sinduri_location = ee.Geometry.Polygon([
    [128.913574, 35.024936],
    [128.913574, 35.090979],
    [128.980522, 35.090979],
    [128.980522, 35.024936],
    [128.913574, 35.024936]
])
# 예제 이미지에 함수 적용 (이미지 ID는 적절하게 변경해야 함)
example_image_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2020-01-01', '2020-03-30')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
    .filterBounds(aoi)

# ImageCollection에서 첫 번째 이미지 선택
example_image = example_image_collection.first()  # 첫 번째 이미지를 선택

processed_image = process_image(example_image)
# 지도 생성 및 시각화
# 처리된 이미지에 수역 마스킹 적용
processed_image = cloud_mask(processed_image)

Map = geemap.Map()
Map.centerObject(aoi, 14)  # 신두리사구를 중심으로 설정

# 육지를 마스킹하기 위한 조건 설정
# water_mask = processed_image.select('UVI').gt(0.5)  # 예시로 0.5보다 큰 값인 픽셀을 수상으로 간주

# 처리된 이미지를 지도에 추가 (수중식물 지수 사용) - 수상만 표시
uvi_params = {
    'bands': ['WEVI'],  # UVI 밴드만 사용
    'min': -0.4, # 수중식물 지수의 최소값
    'max': 1,   # 수중식물 지수의 최대값
    # 'palette': ['purple', 'blue', 'green', 'yellow', 'red']  # 색상 팔레트 설정
    'palette': ['red','yellow','green','blue','purple']  # 색상 팔레트 설정
}
Map.addLayer(processed_image, uvi_params, 'Processed Image (FAI)')

# # 다른 밴드도 추가할 경우 (False Color Composite 사용) - 육지 마스킹 적용
# false_color_params = {
#     'bands': ['B8', 'B4', 'B3'],  # B8, B4, B3 밴드를 사용하여 가짜색 합성
#     'min': 0,
#     'max': 3000
# }
# Map.addLayer(processed_image.updateMask(water_mask), false_color_params, 'Processed Image (False Color - Water Only)')

# 지도를 화면에 표시
Map

NameError: name 'ee' is not defined